# I saved pre processed text in csv file m using that dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk import WordNetLemmatizer
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
urdu_dataset = pd.read_csv("Analysed_Urdu_Tweets.csv",index_col=False)
urdu_dataset.drop(urdu_dataset.columns[0],axis=1,inplace=True)
urdu_dataset.head()


,Tweet,Class
0,ایٹم بم بنایا ھے بھائی ایٹم بمب کوٹ لکھپت اتفا...,2
1,چندے انقلاب عمران خان وزیر اعظم بن سکتے,0
2,ٹویٹر خیال,1
3,سرچ انجن گوگل نائب صدر فضا فٹ بلندی چھلانگ عال...,2
4,اسکی لہریں یار أ,2


In [3]:

## WORDS in dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(urdu_dataset['Tweet'])
sequences = tokenizer.texts_to_sequences(urdu_dataset['Tweet'])

max_sequence_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
## Split the dataset
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, urdu_dataset['Class'], test_size=0.25, random_state=42)

print("Training set shape:",X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (749, 31)
Testing set shape: (250, 31)


# Models

In [4]:
!pip install tensorflow_hub

In [5]:
import gensim.downloader as api
import tensorflow_hub as hub
word2vec_model = api.load("word2vec-google-news-300")
glove_vectors = api.load("glove-twitter-200")
# glove_vectors = api.load("glove-wiki-gigaword-100")
fasttext_model = api.load("fasttext-wiki-news-subwords-300")
elmo = hub.load("https://tfhub.dev/google/elmo/2")

## PARAMETERS DECLARATION

In [16]:
max_length = max([len(seq) for seq in X_train])
X_train_padded = pad_sequences(X_train, maxlen=max_length)
X_test_padded = pad_sequences(X_test, maxlen=max_length)

In [17]:
# word2vec_weights = create_embedding_matrix(word2vec_model, tokenizer)

vocab_size = len(tokenizer.word_index) + 1
embed_matrix = np.zeros((vocab_size,word2vec_model.vector_size))
model_eval=dict()


num_layers =3
dropout_rate=0.7

embedding_dim = 128

for w,i in tokenizer.word_index.items():
    if w in word2vec_model:
        embed_matrix[i] = word2vec_model[w]
word2vec_weights = embed_matrix

## LSTM MODEL

In [18]:
pip install --upgrade keras

In [19]:
# LSTM -> 3,0.7 ->0.66
# nested for for dropout and num_layers

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.25)

Epoch 1/8
18/18 [==============================] - 22s 557ms/step - loss: 0.0000e+00 - accuracy: 0.4688 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 2/8
18/18 [==============================] - 3s 170ms/step - loss: 0.0000e+00 - accuracy: 0.4100 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 3/8
18/18 [==============================] - 2s 138ms/step - loss: 0.0000e+00 - accuracy: 0.4688 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 4/8
18/18 [==============================] - 4s 215ms/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.3191
Epoch 5/8
18/18 [==============================] - 2s 104ms/step - loss: 0.0000e+00 - accuracy: 0.4902 - val_loss: 0.0000e+00 - val_accuracy: 0.2287
Epoch 6/8
18/18 [==============================] - 2s 105ms/step - loss: 0.0000e+00 - accuracy: 0.4884 - val_loss: 0.0000e+00 - val_accuracy: 0.2181
Epoch 7/8
18/18 [==============================] - 2s 125ms/step - loss: 0.0000e+00 - accuracy: 0.4920 - 

In [20]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy = model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 0.2160


In [21]:
model_eval['LSTM(no embedding)'] = {'Precision':precision,'Recall':recall,'F1 score':f1,'accuracy':accuracy}
model_eval

{'LSTM(no embedding)': {'Precision': 0.23621514773200167,
  'Recall': 0.4116575591985428,
  'F1 score': 0.18650082175407137,
  'accuracy': 0.2160000056028366}}

# Word2Vec MODEL

In [22]:
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_weights), output_dim=300,weights=[word2vec_weights], input_length=max_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
10/10 [==============================] - 12s 371ms/step - loss: 0.0000e+00 - accuracy: 0.0735 - val_loss: 0.0000e+00 - val_accuracy: 0.0067
Epoch 2/10
10/10 [==============================] - 2s 182ms/step - loss: 0.0000e+00 - accuracy: 0.3022 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 3/10
10/10 [==============================] - 1s 148ms/step - loss: 0.0000e+00 - accuracy: 0.4958 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 4/10
10/10 [==============================] - 2s 187ms/step - loss: 0.0000e+00 - accuracy: 0.4925 - val_loss: 0.0000e+00 - val_accuracy: 0.3133
Epoch 5/10
10/10 [==============================] - 2s 191ms/step - loss: 0.0000e+00 - accuracy: 0.5025 - val_loss: 0.0000e+00 - val_accuracy: 0.3600
Epoch 6/10
10/10 [==============================] - 2s 194ms/step - loss: 0.0000e+00 - accuracy: 0.5075 - val_loss: 0.0000e+00 - val_accuracy: 0.3067
Epoch 7/10
10/10 [==============================] - 2s 154ms/step - loss: 0.0000e+00 - accuracy: 0.

In [23]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision_word2vec = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall_word2vec = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1_word2vec = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy_word2vec = model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - accuracy: 0.2920


In [24]:
model_eval['LSTM(embedding: word2vec)'] = {'Precision':precision_word2vec,'Recall':recall_word2vec,'F1 score':f1_word2vec,'accuracy':accuracy_word2vec}

## glove

embed matrix

In [30]:
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in glove_vectors.key_to_index:
        embedding_matrix[i] = glove_vectors[word]

ValueError: could not broadcast input array from shape (200,) into shape (100,)

model 

In [29]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(units=3, activation='softmax')) #3 classes
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=0.25)
# model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test,y_test)) i think

Epoch 1/10
12/12 [==============================] - ETA: 0s - loss: 0.9498 - accuracy: 0.4700

IndexError: tuple index out of range

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

8/8 [==============================] - 1s 19ms/step


In [ ]:
###
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)

Evaluate

In [ ]:
# loss, accuracy_glove = model.evaluate(X_test, y_pred)
# precision_glove = precision_score(y_pred, y_pred_labels, average='macro')
# recall_glove = recall_score(y_pred, y_pred_labels, average='macro')
# f1_glove = f1_score(y_pred, y_pred_labels, average='macro')
precision_glove = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall_glove = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1_glove = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy_glove = model.evaluate(X_test, y_test)
model_eval['LSTM(embedding: GLOVE)'] = {'Precision':precision_glove,'Recall':recall_glove,'F1 score':f1_glove,'accuracy':accuracy_glove}

1/8 [==>...........................] - ETA: 0s - loss: 0.7088 - accuracy: 0.5625

8/8 [==============================] - 0s 27ms/step - loss: 0.7918 - accuracy: 0.4960


d:\Python64\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Fast Text

In [ ]:
fasttext_model = api.load("fasttext-wiki-news-subwords-300")

Embed matrix

In [34]:
embed_matrix = np.zeros((vocab_size,fasttext_model.vector_size))

for w,i in tokenizer.word_index.items():
    if w in fasttext_model and w in word2vec_model:
        embed_matrix[i] = word2vec_model[w]
fasttext_weights = embed_matrix    


Model

In [38]:
input_dim = len(fasttext_weights)

input_dim = vocab_size
input_length = max_length
model=Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=300, weights=[embedding_matrix], input_length=input_length, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


ValueError: Layer embedding_7 weight shape (4442, 300) is not compatible with provided weight shape (4442, 100).

In [ ]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)


In [ ]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision_fasttext = precision_score(y_test, y_pred, average='macro')
recall_fasttext = recall_score(y_test, y_pred, average='macro')
f1_fasttext = f1_score(y_test, y_pred, average='macro')
loss, accuracy_fasttext = model.evaluate(X_test, y_test)
model_eval['LSTM(embedding: fast-text)'] = {'Precision':precision_fasttext,'Recall':recall_fasttext,'F1 score':f1_fasttext,'accuracy':accuracy_fasttext}

Elmo

In [ ]:
elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [ ]:
table = pd.DataFrame.from_dict(model_eval, orient='index')
print(table)

                          Precision    Recall  F1 score  accuracy
LSTM(no embedding)         0.162667  0.333333  0.218638       NaN
GRU(embedding: word2vec)   0.162667  0.333333  0.218638       NaN
LSTM(embedding: GLOVE)     0.162667  0.333333  0.218638     0.488
